# Notebook de Cinemática e Dinâmica Simbólica em Python (Colab)

Este notebook replica o fluxo do projeto original em MATLAB usando apenas Python (SymPy). Ele calcula, de forma simbólica, a cinemática direta e as equações de movimento (M, C, G) de um robô em cadeia aberta com quaisquer parâmetros DH e propriedades inerciais fornecidos.

**Destaques**
- Construção 100% simbólica com `sympy`.
- Direções de eixos configuráveis: forneça uma lista de caracteres (por exemplo, `['z', 'y', 'y', 'z', ...]`) para indicar o eixo de rotação ou deslocamento de cada junta.
- Estrutura genérica: suporte a juntas revolutas ou prismáticas, posições de centro de massa arbitrárias e tensores de inércia 3x3.

> Preencha a célula de definição do robô com seus parâmetros (DH, massas, centros de massa e inércia) e o vetor de eixos (x/y/z) — inclusive para modelos ABB/SCARA — e execute as células na ordem.


## 1. Ambiente
Instala apenas o SymPy. O fluxo agora é totalmente simbólico, sem backend numérico ou aceleração GPU/TPU.


In [1]:
!pip -q install sympy > /dev/null


## 2. Importações e utilidades


In [2]:
import sympy as sp
from sympy import Matrix
from dataclasses import dataclass
from typing import List, Literal, Optional, Tuple
from concurrent.futures import ProcessPoolExecutor

sp.init_printing(use_latex='mathjax')


## 3. Estruturas de dados do robô

- `Joint`: tipo (revoluta ou prismática), eixo de movimento (`x`, `y` ou `z`) e índice do parâmetro variável.
- `Link`: parâmetros DH modificados, massa, centro de massa (em coordenadas do elo) e tensor de inércia (no frame do elo).
- `RobotModel`: contém a lista de elos e a gravidade.


In [3]:

JointType = Literal['R', 'P']
AxisType = Literal['x', 'y', 'z']


@dataclass
class Joint:
    joint_type: JointType
    theta: sp.Symbol
    d: sp.Symbol
    a: sp.Symbol
    alpha: sp.Symbol
    axis: AxisType = 'z'  # direção do movimento (rotação ou translação)


@dataclass
class Link:
    joint: Joint
    mass: sp.Symbol
    com: Matrix  # (3, 1)
    inertia: Matrix  # (3, 3) no frame do elo


@dataclass
class RobotModel:
    links: List[Link]
    gravity: Matrix

    @property
    def dof(self) -> int:
        return len(self.links)


## 4. Funções auxiliares

Inclui geração da matriz de transformação homogênea, jacobianos de posição/rotação (agora usando o eixo configurado de cada junta) e composição das transformações ao longo da cadeia.


In [4]:
def dh_transform(theta, d, a, alpha) -> Matrix:
    ct, st = sp.cos(theta), sp.sin(theta)
    ca, sa = sp.cos(alpha), sp.sin(alpha)
    return Matrix([
        [ct, -st * ca, st * sa, a * ct],
        [st, ct * ca, -ct * sa, a * st],
        [0, sa, ca, d],
        [0, 0, 0, 1],
    ])


def axis_from_char(rotation: Matrix, axis: AxisType) -> Matrix:
    idx = {'x': 0, 'y': 1, 'z': 2}[axis]
    return rotation[:, idx]


def validate_axes(axes: List[AxisType], dof: int) -> List[AxisType]:
    if len(axes) != dof:
        raise ValueError(f"Esperam-se {dof} eixos (x/y/z), mas recebi {len(axes)}.")
    invalid = [a for a in axes if a not in ('x', 'y', 'z')]
    if invalid:
        raise ValueError(f"Eixos inválidos: {invalid}. Use apenas x, y ou z.")
    return axes


def parse_axis_order(order: List[str]) -> Tuple[List[JointType], List[AxisType]]:
    joint_types, axes = [], []
    for token in order:
        token_lower = token.lower()
        if token_lower in ('dx', 'dy', 'dz'):
            joint_types.append('P')
            axes.append(token_lower[-1])
        elif token_lower in ('x', 'y', 'z'):
            joint_types.append('R')
            axes.append(token_lower)
        else:
            raise ValueError(
                f"Entrada de eixo inválida: {token}. Use Dx/Dy/Dz para prismáticas ou x/y/z para rotacionais."
            )
    return joint_types, validate_axes(axes, len(order))


def build_links_from_data(
    qs: List[sp.Symbol],
    joint_types: List[JointType],
    axes: List[AxisType],
    dh_params: List[Tuple[sp.Symbol, sp.Symbol, sp.Symbol, sp.Symbol]],
    masses: List[sp.Symbol],
    coms: List[Matrix],
    inertias: List[Matrix],
) -> List[Link]:
    if not (len(qs) == len(joint_types) == len(axes) == len(dh_params) == len(masses) == len(coms) == len(inertias)):
        raise ValueError('Listas de juntas, parâmetros DH, massas, com e inércias devem ter o mesmo tamanho.')

    links = []
    for i, (jt, axis) in enumerate(zip(joint_types, axes)):
        theta_i, d_i, a_i, alpha_i = dh_params[i]
        theta = qs[i] if jt == 'R' else theta_i
        d = qs[i] if jt == 'P' else d_i
        links.append(
            Link(
                Joint(jt, theta, d, a_i, alpha_i, axis=axis),
                mass=masses[i],
                com=coms[i],
                inertia=inertias[i],
            )
        )
    return links


def forward_kinematics(model: RobotModel, debug: bool = False) -> Tuple[List[Matrix], List[Matrix]]:
    Ts, origins = [], [Matrix([0, 0, 0])]
    T = sp.eye(4)
    for idx, link in enumerate(model.links):
        T = T * dh_transform(link.joint.theta, link.joint.d, link.joint.a, link.joint.alpha)
        Ts.append(T)
        origins.append(T[:3, 3])
        if debug:
            print(f"[DEBUG][Cinemática] Elo {idx + 1}/{model.dof} concluído (origem: {origins[-1].T})", flush=True)
    return Ts, origins


def spatial_jacobians(
    model: RobotModel, Ts: List[Matrix], origins: List[Matrix], debug: bool = False
) -> Tuple[List[Matrix], List[Matrix]]:
    motion_axes = []
    for j, link in enumerate(model.links):
        R_prev = sp.eye(3) if j == 0 else Ts[j - 1][:3, :3]
        motion_axes.append(axis_from_char(R_prev, link.joint.axis))

    Jvs, Jws = [], []
    for i, link in enumerate(model.links):
        o_i = origins[i]
        o_com = origins[i + 1] + Ts[i][:3, :3] * link.com
        Jv_cols, Jw_cols = [], []
        for j in range(model.dof):
            axis_vec = motion_axes[j]
            o_j = origins[j]
            if model.links[j].joint.joint_type == 'R':
                Jv_cols.append(axis_vec.cross(o_com - o_j))
                Jw_cols.append(axis_vec)
            else:  # prismática
                Jv_cols.append(axis_vec)
                Jw_cols.append(Matrix([0, 0, 0]))
        Jvs.append(Matrix.hstack(*Jv_cols))
        Jws.append(Matrix.hstack(*Jw_cols))
        if debug:
            print(f"[DEBUG][Cinemática] Jacobianos do elo {i + 1}/{model.dof} calculados.", flush=True)
    return Jvs, Jws


def _energy_terms(args):
    link, Jv_i, Jw_i, origin_next, T_i, dq_vec, gravity = args
    R = T_i[:3, :3]
    v = Jv_i * dq_vec
    w = Jw_i * dq_vec
    I_world = R * link.inertia * R.T
    kinetic = 0.5 * link.mass * (v.T * v)[0] + 0.5 * (w.T * I_world * w)[0]
    potential = link.mass * gravity.dot(origin_next + R * link.com)
    return kinetic, potential


def _lagrange_tau_term(args):
    L, qs, dqs, ddqs, idx = args
    q, dq, ddq = qs[idx], dqs[idx], ddqs[idx]
    dL_dq = sp.diff(L, q)
    dL_ddq = sp.diff(L, dq)
    d_dt_dL_ddq = sum(sp.diff(dL_ddq, q_var) * dq_var for q_var, dq_var in zip(qs, dqs))
    d_dt_dL_ddq += sum(sp.diff(dL_ddq, dq_var) * ddq_var for dq_var, ddq_var in zip(dqs, ddqs))
    return d_dt_dL_ddq - dL_dq


## 5. Energia, matrizes M/C/G e equações de movimento

Calculamos a energia cinética/potencial de cada elo e aplicamos Lagrange para obter `M(q)`, `C(q, q̇)` e `G(q)`, bem como o vetor de torques/forças `\tau`.


In [5]:
def dynamics(
    model: RobotModel,
    qs: List[sp.Symbol],
    dqs: List[sp.Symbol],
    ddqs: List[sp.Symbol],
    parallel: bool = False,
    processes: Optional[int] = None,
    debug: bool = False,
):
    Ts, origins = forward_kinematics(model, debug=debug)
    Jvs, Jws = spatial_jacobians(model, Ts, origins, debug=debug)

    dq_vec = Matrix(dqs)
    energy_args = [
        (link, Jvs[i], Jws[i], origins[i + 1], Ts[i], dq_vec, model.gravity)
        for i, link in enumerate(model.links)
    ]

    energy_results = []
    if parallel and model.dof > 1:
        with ProcessPoolExecutor(max_workers=processes) as executor:
            for idx, res in enumerate(executor.map(_energy_terms, energy_args), 1):
                energy_results.append(res)
                if debug:
                    print(
                        f"[DEBUG][Dinâmica] Energias do elo {idx}/{model.dof} calculadas.",
                        flush=True,
                    )
    else:
        for idx, arg in enumerate(energy_args, 1):
            energy_results.append(_energy_terms(arg))
            if debug:
                print(
                    f"[DEBUG][Dinâmica] Energias do elo {idx}/{model.dof} calculadas.",
                    flush=True,
                )

    kinetic_terms, potential_terms = zip(*energy_results)
    L = sp.together(sp.Add(*kinetic_terms) - sp.Add(*potential_terms))

    tau_args = [(L, qs, dqs, ddqs, idx) for idx in range(model.dof)]
    tau_terms = []
    if parallel and model.dof > 1:
        with ProcessPoolExecutor(max_workers=processes) as executor:
            for idx, res in enumerate(executor.map(_lagrange_tau_term, tau_args), 1):
                tau_terms.append(res)
                if debug:
                    print(
                        f"[DEBUG][Dinâmica] Equação de torque τ{idx} derivada.",
                        flush=True,
                    )
    else:
        for idx, arg in enumerate(tau_args, 1):
            tau_terms.append(_lagrange_tau_term(arg))
            if debug:
                print(
                    f"[DEBUG][Dinâmica] Equação de torque τ{idx} derivada.",
                    flush=True,
                )

    tau_raw = Matrix(tau_terms)
    M_raw = tau_raw.jacobian(ddqs)
    zero_dd = {dd: 0 for dd in ddqs}
    zero_d = {dq: 0 for dq in dqs}
    Cg_raw = tau_raw.xreplace(zero_dd)
    C_raw = Cg_raw - Cg_raw.xreplace(zero_d)
    G_raw = Cg_raw.xreplace(zero_d)

    # Otimiza a forma simbólica com CSE para reduzir subexpressões repetidas
    replacements, reduced_exprs = sp.cse([M_raw, C_raw, G_raw, tau_raw], optimizations='basic')
    M_opt, C_opt, G_opt, tau_opt = (sp.Matrix(expr) if hasattr(expr, 'shape') else expr for expr in reduced_exprs)
    return replacements, M_opt, C_opt, G_opt, tau_opt


## 6. Exemplo: manipulador de 3 DOF (revoluto)

Preencha a célula abaixo com os parâmetros do seu robô e o vetor de eixos (ex.: `['z', 'y', 'y']`). Para usar os dados do ABB ou SCARA, basta adaptar a lista `links` com os DH, massas, centros de massa e inércias exportados do MATLAB.


In [6]:

# Variáveis generalizadas
q1, q2, q3 = sp.symbols('q1 q2 q3', real=True)
dq1, dq2, dq3 = sp.symbols('dq1 dq2 dq3', real=True)
ddq1, ddq2, ddq3 = sp.symbols('ddq1 ddq2 ddq3', real=True)

# Parâmetros DH (theta, d, a, alpha) com juntas revolutas
# Escolha o eixo de cada junta (rotacional ou prismática) com caracteres x/y/z
axes = validate_axes(['z', 'y', 'y'], 3)  # ex.: ['z','y','y','z','y','z'] para 6 DOF

# Assumptions físicas
params = sp.symbols(
    'a1 a2 a3 d1 d2 d3 alpha1 alpha2 alpha3 '
    'm1 m2 m3 lx2 lx3 lz1 '
    'Ixx1 Iyy1 Izz1 Ixx2 Iyy2 Izz2 Ixx3 Iyy3 Izz3 g',
    real=True,
)
(
    a1, a2, a3,
    d1, d2, d3,
    alpha1, alpha2, alpha3,
    m1, m2, m3,
    lx2, lx3, lz1,
    Ixx1, Iyy1, Izz1,
    Ixx2, Iyy2, Izz2,
    Ixx3, Iyy3, Izz3,
    g,
) = params

m1, m2, m3 = [p for p in (m1, m2, m3)]
Izz1, Izz2, Izz3 = [sp.symbols(n, real=True, positive=True) for n in ['Izz1', 'Izz2', 'Izz3']]

link1 = Link(
    Joint('R', q1, d1, a1, alpha1, axis=axes[0]),
    mass=m1,
    com=Matrix([0, 0, lz1]),
    inertia=sp.diag(Ixx1, Iyy1, Izz1),
)

link2 = Link(
    Joint('R', q2, d2, a2, alpha2, axis=axes[1]),
    mass=m2,
    com=Matrix([lx2, 0, 0]),
    inertia=sp.diag(Ixx2, Iyy2, Izz2),
)

link3 = Link(
    Joint('R', q3, d3, a3, alpha3, axis=axes[2]),
    mass=m3,
    com=Matrix([lx3, 0, 0]),
    inertia=sp.diag(Ixx3, Iyy3, Izz3),
)

robot = RobotModel(
    links=[link1, link2, link3],
    gravity=Matrix([0, 0, -g]),
)

cse_defs, M, C, G, tau = dynamics(robot, [q1, q2, q3], [dq1, dq2, dq3], [ddq1, ddq2, ddq3])

print('Subexpressões comuns (CSE):')
for sym, expr in cse_defs:
    print(f"  {sym} = {expr}")

print('Matriz de inércia M(q):')
sp.pprint(M)

print('Matriz de Coriolis/Centrífuga C(q,q̇):')
sp.pprint(C)

print('Vetor gravitacional G(q):')
sp.pprint(G)

print('Equações de movimento τ = M·q̈ + C + G:')
sp.pprint(tau)


Subexpressões comuns (CSE):
  x0 = sin(q1)
  x1 = a1*x0
  x2 = cos(q2)
  x3 = x0*x2
  x4 = a2*x3
  x5 = cos(q1)
  x6 = sin(alpha1)
  x7 = d2*x6
  x8 = x5*x7
  x9 = cos(alpha1)
  x10 = sin(q2)
  x11 = x10*x5
  x12 = x11*x9
  x13 = a2*x12
  x14 = x13 + x4 - x8
  x15 = cos(q3)
  x16 = x12 + x3
  x17 = x15*x16
  x18 = sin(q3)
  x19 = sin(alpha2)
  x20 = x5*x6
  x21 = cos(alpha2)
  x22 = x0*x10
  x23 = x21*x9
  x24 = x2*x5
  x25 = x19*x20 + x21*x22 - x23*x24
  x26 = x18*x25
  x27 = x17 - x26
  x28 = x19*x9
  x29 = -x19*x22 + x20*x21 + x24*x28
  x30 = d3*x29
  x31 = a3*x17
  x32 = a3*x26
  x33 = -x30 + x31 - x32
  x34 = lx3*x27 + x33
  x35 = x14 + x34
  x36 = x1 + x35
  x37 = x0*x6
  x38 = x11*x19 + x21*x37 + x28*x3
  x39 = d3*x38
  x40 = x22*x9
  x41 = -x24 + x40
  x42 = x15*x41
  x43 = x11*x21 - x19*x37 + x23*x3
  x44 = x18*x43
  x45 = x42 + x44
  x46 = lx3*x45
  x47 = a3*x42
  x48 = a3*x44
  x49 = a1*x5
  x50 = a2*x24
  x51 = x0*x7
  x52 = a2*x40
  x53 = x50 + x51 - x52
  x54 = x49 + x53


## 7. Exemplo: UVMS (12 DOF)

Configuração com 6 graus de liberdade do ROV (3 translações + 3 rotações) seguidos de 6 juntas do braço. Use `matriz_excentricidades` para posicionar o centro de massa de cada junta e `ordem` para indicar se cada movimento é prismático (Dx/Dy/Dz) ou rotacional (x/y/z).


Observações:
- A cinemática direta usa apenas os parâmetros DH (sem deslocar por excentricidades); os quadros e o TCP seguem o referencial DH.
- As excentricidades da `matriz_excentricidades` são aplicadas aos centros de massa de *todos* os elos na etapa dinâmica (M/C/G e τ), não só no último elo.


In [9]:
import time
import sys

print("--- INÍCIO DO TESTE ---", flush=True)

for i in range(5):
    # Use sys.stdout.flush() como uma alternativa mais forte ao flush=True
    sys.stdout.write(f"Etapa {i} executada em tempo real...\n")
    sys.stdout.flush()
    time.sleep(1)

print("--- FIM DO TESTE ---", flush=True)

--- INÍCIO DO TESTE ---
Etapa 0 executada em tempo real...
Etapa 1 executada em tempo real...
Etapa 2 executada em tempo real...
Etapa 3 executada em tempo real...
Etapa 4 executada em tempo real...
--- FIM DO TESTE ---


In [7]:

# Ordem das juntas: 3 translações + 3 rotações do ROV + 6 rotações do braço
ordem = ['Dx', 'Dy', 'Dz', 'z', 'y', 'x', 'z', 'y', 'y', 'z', 'y', 'z']
joint_types, axes = parse_axis_order(ordem)

# Variáveis generalizadas
qs_uvms = list(sp.symbols('q1:13', real=True))
dqs_uvms = list(sp.symbols('dq1:13', real=True))
ddqs_uvms = list(sp.symbols('ddq1:13', real=True))

# Parâmetros DH genéricos (use 12 tuplas de theta, d, a, alpha conforme seu modelo)
theta_syms = sp.symbols('theta1:13', real=True)
d_syms = sp.symbols('d1:13', real=True)
a_syms = sp.symbols('a1:13', real=True)
alpha_syms = sp.symbols('alpha1:13', real=True)
dh_params_uvms = list(zip(theta_syms, d_syms, a_syms, alpha_syms))

# Centros de massa (matriz de excentricidades 12x3)
matriz_excentricidades = Matrix([
    [0, 0, 0],  # 1) Dx
    [0, 0, 0],  # 2) Dy
    [0, 0, 0],  # 3) Dz
    [0, 0, 0],  # 4) Rz (yaw)
    [0, 0, 0],  # 5) Ry (pitch)
    [0, 0, 0],  # 6) Rx (roll)
    [0, 0, 1],  # 7) p1: pedestal -> eixo 2
    [1, 0, 0],  # 8) p2: elo 1 -> elo 2
    [1, 0, 0],  # 9) p3: elo 2 -> elo 3
    [1, 0, 0],  # 10) p4: elo 3 -> wrist center
    [0, 0, 0],  # 11) p5: punho esférico
    [0, 0, 1],  # 12) p6: wrist center -> TCP
])
coms_uvms = [Matrix(matriz_excentricidades[i, :]).reshape(3, 1) for i in range(matriz_excentricidades.rows)]

# Massas e tensores de inércia
m_syms = list(sp.symbols('m1:13', real=True, positive=True))
inertias_uvms = [
    sp.diag(*sp.symbols(f'Ixx{i+1} Iyy{i+1} Izz{i+1}', real=True, positive=True))
    for i in range(len(ordem))
]

g = sp.symbols('g', real=True)
links_uvms = build_links_from_data(qs_uvms, joint_types, axes, dh_params_uvms, m_syms, coms_uvms, inertias_uvms)
uvms = RobotModel(links=links_uvms, gravity=Matrix([0, 0, -g]))

# Execute com parallel=True para aproveitar múltiplos núcleos no cálculo simbólico
# (pode ser pesado para 12 DOF; ajuste processes conforme sua máquina)
cse_uvms, M_uvms, C_uvms, G_uvms, tau_uvms = dynamics(
    uvms,
    qs_uvms,
    dqs_uvms,
    ddqs_uvms,
    parallel=True,
    processes=None,
)

print('UVMS: subexpressões comuns (CSE)')
for sym, expr in cse_uvms[:5]:  # mostra só as primeiras para evitar saída enorme
    print(f"  {sym} = {expr}")
print('Tamanho de M(q):', M_uvms.shape)
print('Tamanho de C(q,q̇):', C_uvms.shape)
print('Tamanho de G(q):', G_uvms.shape)
print('Tamanho de τ:', tau_uvms.shape)



KeyboardInterrupt: 

## 8. Exportação simbólica
Use as subexpressões (CSE) para gerar código limpo para Simulink/S-Function:
- `sympy.ccode` ou `sympy.printing` para exportar M, C, G e τ.
- Aproveite a lista de `cse_defs` para reutilizar subexpressões e evitar recomputo na implementação.
